# labelmeでアノテーションした画像を分割するプログラム


# Library

In [1]:
import os
import glob
import json
import numpy as np
import pandas as pd

from PIL import Image
import matplotlib.pyplot as plt
import cv2

import tqdm


In [ ]:
label2index = {
    'apple': 0,
    'banana': 1
}

index2label = {
    0: 'apple',
    1: 'banana'
}

# define "ImageDevider_labelme2YOLO" class

In [6]:
# 学習用（labelmeのアノテーションファイルが存在する場合）
class ImageDevider_labelme2YOLO():
    def __init__(self, out_y, out_x, overlap, output_img_folder, output_yolo_folder) -> None:
        self.out_y = out_y
        self.out_x = out_x
        self.overlap = overlap
        self.output_img_folder = output_img_folder
        self.output_yolo_folder = output_yolo_folder

    def devide_image(self, image_file, is_train=False):
        # create json file name
        basename = os.path.basename(image_file).split('.')[0]
        basepath = os.path.dirname(image_file)
        labelme_path = f"{basepath}/{basename}.json"

        img = Image.open(image_file)
        img_array = np.array(img)
        in_y, in_x, _ = img_array.shape

        num_images_y = in_y // self.out_y
        num_images_x = in_x // self.out_x

        # if crop images are lack at righet & bottom, num of image +1
        if in_y - num_images_y * (self.out_y - self.overlap) > self.overlap:
            num_images_y += 1
        if in_x- num_images_x * (self.out_x - self.overlap) > self.overlap:
            num_images_x += 1

        # set index to rename save-file name , when save crop image and YOLO.txt
        index = 0
        for i in range(num_images_y):
            for j in range(num_images_x):
                start_y = np.maximum(0, i * (self.out_y - self.overlap))
                start_x = np.maximum(0, j * (self.out_x - self.overlap))
                end_y = i * (self.out_y - self.overlap) + self.out_y
                end_x = j * (self.out_x - self.overlap) + self.out_x
                output_array = img_array[start_y: end_y, start_x: end_x]
                
                img = cv2.copyMakeBorder(output_array, 
                                   0, self.out_y - output_array.shape[0], 
                                   0, self.out_x - output_array.shape[1], 
                                   cv2.BORDER_CONSTANT, 
                                   0)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

                yolo_list = [] # strage anotation data / image
                # get labelme data to YOLO
                if os.path.exists(labelme_path):
                    with open(labelme_path, 'r') as f:
                        labelme_data = json.load(f)
                    for shape in labelme_data['shapes']:
                        _yolo_list = []
                        # get point datas eg.[[start x, start y],[end x, end y]]
                        for point in shape['points']:
                            x, y = point # x, yの順
                            # over 0 and under crop size
                            crop_y = np.minimum(self.out_y, np.maximum(0, y - start_y))
                            crop_x = np.minimum(self.out_x, np.maximum(0, x - start_x))
                            _yolo_list.append(crop_x / self.out_x) # x y の順
                            _yolo_list.append(crop_y / self.out_y)
                        sx, sy, ex, ey = _yolo_list
                        if sx != ex and sy != ey:
                            yolo_list.append([label2index[shape['label']], 
                                            (sx + ex) / 2, 
                                            (sy + ey) / 2, 
                                            (ex - sx), 
                                            (ey - sy)])

                # save image and YOLOtxt
                if is_train:
                    if len(yolo_list) != 0:
                        yolo_df = pd.DataFrame(yolo_list)
                        yolo_df.to_csv(self.output_yolo_folder + f'{basename}_{index:02}.txt', sep=(' '), header=None, index=None)
                        cv2.imwrite(self.output_img_folder + f'{basename}_{index:02}.png', img)
                else:
                    cv2.imwrite(self.output_img_folder + f'{basename}_{index:02}.png', img)
                    if len(yolo_list) != 0:
                        yolo_df = pd.DataFrame(yolo_list)
                        yolo_df.to_csv(self.output_yolo_folder + f'{basename}_{index:02}.txt', sep=(' '), header=None, index=None)

                index += 1



# 使用例

In [ ]:
output_image_folder = './output/images/'
output_YOLO_folder = './output/labels/'

labelme_file = './data/images/000.json' # 同じフォルダに000.jpegなど画像ファイルが存在

In [7]:

crop_man_for_training = ImageDevider_labelme2YOLO(1280, 1280, 256, output_image_folder, output_YOLO_folder)

In [9]:
crop_man_for_training.devide_image(labelme_file)